In [13]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [14]:
background = None

accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [15]:
def calc_accumulated_avg(frame,accumulated_weight):
    global background
    
    if background is None:
        background =frame.copy().astype('float32')
        return None
    
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [16]:
def segment(frame, threshold_min=25):
    
    #absolute difference
    diff = cv2.absdiff(background.astype('uint8'), frame)
    
    ret, thresholded = cv2.threshold(diff, threshold_min,255, cv2.THRESH_BINARY)
    
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) ==0:
        return None
    else:
        #assuming the largest contour in ROI is hand
        hand_segment = max(contours, key=cv2.contourArea)
        
    return (thresholded, hand_segment)

In [17]:
def count_fingers(thresholded, hand_segment):
    
    #convex Hull
    conv_hull = cv2.convexHull(hand_segment)
    
    #TOP (x,y)
    top = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0]) 
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0]) # (x,y)
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()] [0]) #(x,y)
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()] [0]) #(x,y)
    
    cX = (left[0] + right[0]) // 2 # provides int form
    cY = (top[1] + bottom[1]) // 2 
    
    #euclidean distance from center to left, right, top and bottom
    distance = pairwise.euclidean_distances([(cX,cY)], Y=[left, right, top, bottom])[0]
    
    max_distance = distance.max()
    
    radius = int (0.9 * max_distance) # 90 percent of the distance
    circumference = (2*np.pi*radius)
    
    #ciruclar roi
    cir_roi = np.zeros(thresholded.shape[:2], dtype='uint8')
    
    cv2.circle(cir_roi, (cX,cY), radius, 255, 10)
    
    cir_roi = cv2.bitwise_and(thresholded, thresholded, mask = cir_roi)
    
    contours, hierarchy = cv2.findContours (cir_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    count =0
    
    for cont in contours:
        
        (x,y,w,h) = cv2.boundingRect(cont) # ???
        
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
        
        lim_point = ((circumference*0.25) > cont.shape[0])
        
        if out_of_wrist and lim_point:
            count +=1
            
    return count


In [18]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    ret, frame = cam.read()
    
    frame_cpy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    blur_gray = cv2.GaussianBlur(roi_gray, (7,7), 0)
    
    if num_frames <60:
        calc_accumulated_avg(blur_gray,accumulated_weight)
        
        if num_frames <=59:
            cv2.putText(frame_cpy, 'Wait. Getting Background', (200,350), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('Finger_Count', frame_cpy)
        
    else:
        hand = segment(blur_gray)
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            #draws contours around the hand on screen
            cv2.drawContours(frame_cpy, [hand_segment+(roi_right,roi_top)], -1, (255,0,0),5)
            
            fingers = count_fingers(thresholded, hand_segment)
            
            cv2.putText(frame_cpy, str(fingers), (70,50), cv2.FONT_HERSHEY_SIMPLEX,1, (255,0,0),2)
            
            cv2.imshow('Thresholded', thresholded)
            
    cv2.rectangle(frame_cpy, (roi_left,roi_top), (roi_right, roi_bottom),(255,0,0), 5)
    
    num_frames +=1
    
    cv2.imshow("Finger_Count", frame_cpy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k==27:
        break
        
cam.release()
cv2.destroyAllWindows()
        